# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

In [2]:
# Import API key
from api_keys import g_key

In [3]:
target_city = "Los Angeles"
target_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={target_city}&key={g_key}"

In [4]:
geo_data = requests.get(target_url).json()

In [5]:
print(geo_data)

{'results': [{'address_components': [{'long_name': 'Los Angeles', 'short_name': 'Los Angeles', 'types': ['locality', 'political']}, {'long_name': 'Los Angeles County', 'short_name': 'Los Angeles County', 'types': ['administrative_area_level_2', 'political']}, {'long_name': 'California', 'short_name': 'CA', 'types': ['administrative_area_level_1', 'political']}, {'long_name': 'United States', 'short_name': 'US', 'types': ['country', 'political']}], 'formatted_address': 'Los Angeles, CA, USA', 'geometry': {'bounds': {'northeast': {'lat': 34.3373061, 'lng': -118.1552891}, 'southwest': {'lat': 33.7036519, 'lng': -118.6681759}}, 'location': {'lat': 34.0522342, 'lng': -118.2436849}, 'location_type': 'APPROXIMATE', 'viewport': {'northeast': {'lat': 34.3373061, 'lng': -118.1552891}, 'southwest': {'lat': 33.7036519, 'lng': -118.6681759}}}, 'place_id': 'ChIJE9on3F3HwoAR9AhGJW_fL-I', 'types': ['locality', 'political']}], 'status': 'OK'}


In [6]:
print(json.dumps(geo_data, indent=4, sort_keys=True))

{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "Los Angeles",
                    "short_name": "Los Angeles",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
                {
                    "long_name": "Los Angeles County",
                    "short_name": "Los Angeles County",
                    "types": [
                        "administrative_area_level_2",
                        "political"
                    ]
                },
                {
                    "long_name": "California",
                    "short_name": "CA",
                    "types": [
                        "administrative_area_level_1",
                        "political"
                    ]
                },
                {
                    "long_name": "United States",
                    "short_name": "US",
        

In [7]:
lat = geo_data["results"][0]["geometry"]["location"]["lat"]
lng = geo_data["results"][0]["geometry"]["location"]["lng"]
print('''
        City: {0}
        Latitude: {1}
        Longitude: {2}
        '''.format(target_city, lat, lng))


        City: Los Angeles
        Latitude: 34.0522342
        Longitude: -118.2436849
        


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [8]:
cities_temperature = "output_data/clean_city_data.csv"

In [9]:
cities_temperature = pd.read_csv("output_data/clean_city_data.csv")
cities_temperature

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,East London,-33.0153,27.9116,67.14,63,0,4.61,ZA,1659450570
1,1,Upernavik,72.7868,-56.1549,45.25,80,87,11.54,GL,1659450570
2,2,Rikitea,-23.1203,-134.9692,72.95,72,83,21.81,PF,1659450571
3,4,Kharan,28.5833,65.4167,102.67,15,15,2.08,PK,1659450572
4,5,Agüimes,27.9054,-15.4461,83.03,54,20,29.93,ES,1659449997
...,...,...,...,...,...,...,...,...,...,...
562,568,Taltal,-25.4000,-70.4833,58.82,72,27,3.80,CL,1659450750
563,569,Caravelas,-17.7125,-39.2481,77.50,64,77,6.17,BR,1659450823
564,570,Wieluń,51.2210,18.5696,75.25,54,76,6.42,PL,1659450823
565,571,Lidköping,58.5052,13.1577,61.41,76,100,8.55,SE,1659450824


In [10]:
del cities_temperature['Unnamed: 0']
cities_temperature.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,East London,-33.0153,27.9116,67.14,63,0,4.61,ZA,1659450570
1,Upernavik,72.7868,-56.1549,45.25,80,87,11.54,GL,1659450570
2,Rikitea,-23.1203,-134.9692,72.95,72,83,21.81,PF,1659450571
3,Kharan,28.5833,65.4167,102.67,15,15,2.08,PK,1659450572
4,Agüimes,27.9054,-15.4461,83.03,54,20,29.93,ES,1659449997


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
gmaps.configure(api_key=g_key)

In [12]:
locations = cities_temperature[["Lat", "Lng"]].astype(float)
weights = cities_temperature["Humidity"].astype(float)

In [13]:
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
cities_humid = cities_temperature["Humidity"].tolist()

In [15]:
map_humid_layer = gmaps.symbol_layer(locations, fill_color='rgba(0,150,0,0.4)',
                                    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
                                    info_box_content=[f"Humidity: {humidity}" for humidity in cities_humid])
fig = gmaps.figure()
fig.add_layer(map_humid_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [16]:
fig = gmaps.figure()
fig.add_layer(heat_layer)
fig.add_layer(map_humid_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [17]:
humidity_mean = round(cities_temperature["Humidity"].astype(float).mean(), 2)
humidity_median = round(cities_temperature["Humidity"].astype(float).median(), 2)
humidity_mode = round(cities_temperature["Humidity"].astype(float).mode(), 2)

print(f"Humidity Mean:{humidity_mean}")
print(f"Humidity Median:{humidity_median}")
print(f"Humidity Mode:{humidity_mode}")

Humidity Mean:68.28
Humidity Median:72.0
Humidity Mode:0    94.0
Name: Humidity, dtype: float64


In [18]:
narrow_city = cities_temperature[(cities_temperature["Max Temp"] > 65) & (cities_temperature["Max Temp"] < 75)]
narrow_city.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,East London,-33.0153,27.9116,67.14,63,0,4.61,ZA,1659450570
2,Rikitea,-23.1203,-134.9692,72.95,72,83,21.81,PF,1659450571
6,Baraboo,43.4711,-89.7443,73.54,81,0,4.61,US,1659450544
13,Sioux Lookout,50.1001,-91.9170,66.27,88,100,4.61,CA,1659450288
18,Vaini,-21.2000,-175.2000,73.56,94,40,6.91,TO,1659450576


In [19]:
less_humidity_city = narrow_city[(narrow_city["Humidity"] < 68)]
less_humidity_city.head(41)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,East London,-33.0153,27.9116,67.14,63,0,4.61,ZA,1659450570
24,Avarua,-21.2078,-159.7750,73.45,60,40,8.05,CK,1659450578
43,Carnarvon,-24.8667,113.6333,66.58,63,90,18.21,AU,1659450061
44,Poli,-3.3333,36.8000,70.74,38,43,8.70,TZ,1659450584
49,Faya,18.3851,42.4509,69.91,64,98,4.45,SA,1659450020
53,Castro,-24.7911,-50.0119,72.79,44,0,2.19,BR,1659450587
107,Port Elizabeth,-33.9180,25.5701,67.32,51,0,4.61,ZA,1659450023
113,San Cristobal,7.7669,-72.2250,73.58,59,96,6.55,VE,1659450615
118,Fallon,39.4735,-118.7774,74.44,47,0,6.91,US,1659450618
121,Kazygurt,41.7589,69.3864,73.33,53,84,9.51,KZ,1659450619


In [20]:
less_humidity_city.shape

(41, 9)

In [21]:
less_humidity_city = narrow_city.dropna()

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [22]:
target_coordinates = ", -159.7750"

In [23]:
test_latlng = f'{less_humidity_city.iloc[0]["Lat"]},{less_humidity_city.iloc[0]["Lng"]}'
test_latlng

'-33.0153,27.9116'

In [24]:
hotels_df = less_humidity_city.copy()
hotels_df["Hotel"] = ""
hotels_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
0,East London,-33.0153,27.9116,67.14,63,0,4.61,ZA,1659450570,
2,Rikitea,-23.1203,-134.9692,72.95,72,83,21.81,PF,1659450571,
6,Baraboo,43.4711,-89.7443,73.54,81,0,4.61,US,1659450544,
13,Sioux Lookout,50.1001,-91.9170,66.27,88,100,4.61,CA,1659450288,
18,Vaini,-21.2000,-175.2000,73.56,94,40,6.91,TO,1659450576,
...,...,...,...,...,...,...,...,...,...,...
550,Stromness,58.9650,-3.2960,67.96,83,88,8.99,GB,1659450817,
552,Drovyanaya,51.5833,113.0333,65.17,79,100,2.15,RU,1659450818,
555,Richards Bay,-28.7830,32.0377,68.54,69,0,15.39,ZA,1659450819,
556,Mareeba,-17.0000,145.4333,65.12,95,44,2.66,AU,1659450820,


In [25]:
parameters = {
    "radius":5000,
    "type":"lodging",
    "key":g_key
}

nearby_search_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for idx, row in hotels_df.iterrows():
    lat_lng = f'{row["Lat"]},{row["Lng"]}'
    parameters["location"] = lat_lng
    
    try:
        hotel_res = requests.get(nearby_search_url, params=parameters).json()
        hotels_df.loc[idx, "Hotel"] = hotel_res["results"][0]["name"]
        print("Added hotel for" + row["City"])
    except:
        print("Could not find hotel for" + row["City"])
    

hotels_df





Added hotel forEast London
Added hotel forRikitea
Added hotel forBaraboo
Added hotel forSioux Lookout
Added hotel forVaini
Added hotel forAvarua
Added hotel forBambous Virieux
Added hotel forNikolskoye
Added hotel forHilo
Added hotel forJamestown
Added hotel forCarnarvon
Added hotel forPoli
Added hotel forPortland
Added hotel forFaya
Added hotel forCastro
Added hotel forMahébourg
Could not find hotel forVostok
Added hotel forBerlevåg
Added hotel forAjra
Could not find hotel forKrasnoyarskiy
Added hotel forPuerto Ayora
Added hotel forSouillac
Added hotel forPort Hedland
Added hotel forTanete
Added hotel forPort Elizabeth
Added hotel forSan Cristobal
Added hotel forAlofi
Added hotel forFallon
Added hotel forKazygurt
Could not find hotel forVerkhnyaya Inta
Added hotel forKruisfontein
Added hotel forYulara
Added hotel forPort Alfred
Could not find hotel forOstrovnoy
Added hotel forAksha
Added hotel forQuatre Cocos
Added hotel forLagoa
Added hotel forMondlo
Added hotel forPohrebyshche
Added

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
0,East London,-33.0153,27.9116,67.14,63,0,4.61,ZA,1659450570,Tu Casa
2,Rikitea,-23.1203,-134.9692,72.95,72,83,21.81,PF,1659450571,People ThankYou
6,Baraboo,43.4711,-89.7443,73.54,81,0,4.61,US,1659450544,Thunderbird Motor Inn
13,Sioux Lookout,50.1001,-91.9170,66.27,88,100,4.61,CA,1659450288,Knobby's Fly-In Camps
18,Vaini,-21.2000,-175.2000,73.56,94,40,6.91,TO,1659450576,Keleti Beach Resort
...,...,...,...,...,...,...,...,...,...,...
550,Stromness,58.9650,-3.2960,67.96,83,88,8.99,GB,1659450817,The Old Smokery
552,Drovyanaya,51.5833,113.0333,65.17,79,100,2.15,RU,1659450818,
555,Richards Bay,-28.7830,32.0377,68.54,69,0,15.39,ZA,1659450819,BON Hotel Waterfront Richards Bay
556,Mareeba,-17.0000,145.4333,65.12,95,44,2.66,AU,1659450820,Trinity Plains Tourist Park


In [26]:
clean_hotels_df = hotels_df[hotels_df["Hotel"] != ""]
clean_hotels_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel
0,East London,-33.0153,27.9116,67.14,63,0,4.61,ZA,1659450570,Tu Casa
2,Rikitea,-23.1203,-134.9692,72.95,72,83,21.81,PF,1659450571,People ThankYou
6,Baraboo,43.4711,-89.7443,73.54,81,0,4.61,US,1659450544,Thunderbird Motor Inn
13,Sioux Lookout,50.1001,-91.9170,66.27,88,100,4.61,CA,1659450288,Knobby's Fly-In Camps
18,Vaini,-21.2000,-175.2000,73.56,94,40,6.91,TO,1659450576,Keleti Beach Resort
...,...,...,...,...,...,...,...,...,...,...
516,Goya,-29.1400,-59.2626,68.95,64,51,6.29,AR,1659450800,Apart Costanera Goya
550,Stromness,58.9650,-3.2960,67.96,83,88,8.99,GB,1659450817,The Old Smokery
555,Richards Bay,-28.7830,32.0377,68.54,69,0,15.39,ZA,1659450819,BON Hotel Waterfront Richards Bay
556,Mareeba,-17.0000,145.4333,65.12,95,44,2.66,AU,1659450820,Trinity Plains Tourist Park


In [27]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in clean_hotels_df.iterrows()]
locations = clean_hotels_df[["Lat", "Lng"]]
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Tu Casa</dd>\n<dt>City</dt><dd>East London</dd>\n<dt>Country</dt><dd>ZA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>People ThankYou</dd>\n<dt>City</dt><dd>Rikitea</dd>\n<dt>Country</dt><dd>PF</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Thunderbird Motor Inn</dd>\n<dt>City</dt><dd>Baraboo</dd>\n<dt>Country</dt><dd>US</dd>\n</dl>\n',
 "\n<dl>\n<dt>Name</dt><dd>Knobby's Fly-In Camps</dd>\n<dt>City</dt><dd>Sioux Lookout</dd>\n<dt>Country</dt><dd>CA</dd>\n</dl>\n",
 '\n<dl>\n<dt>Name</dt><dd>Keleti Beach Resort</dd>\n<dt>City</dt><dd>Vaini</dd>\n<dt>Country</dt><dd>TO</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Paradise Inn</dd>\n<dt>City</dt><dd>Avarua</dd>\n<dt>Country</dt><dd>CK</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Casa Tia Villa</dd>\n<dt>City</dt><dd>Bambous Virieux</dd>\n<dt>Country</dt><dd>MU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Tourist House - Sablino</dd>\n<dt>City</dt><dd>Nikolskoye</dd>\n<dt>Country</dt><dd>RU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name

In [28]:
# Add marker layer ontop of heat map
fig = gmaps.figure()
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))